In [19]:
import os
input_image_path = "../imagen/rgbImagen.tif"
output_folder = "../output_blocksv3"
cmd =f"gdal_translate -ot Float32 {input_image_path} ../imagen/rgbImagen32.tif"
os.system(cmd)

Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [21]:
import os
import rasterio
from rasterio.windows import Window

def cut_image_into_blocks(image_path, output_folder):
    src = rasterio.open(image_path)
    width = src.width
    height = src.height
    # Lectura del valor mínimo de la banda 1
    arrayMin1 = src.read(1)
    banda1Min = arrayMin1.min()
    
    # Lectura del valor máximo de la banda 1
    arrayMax1 = src.read(1)
    banda1Max = arrayMax1.max()

    # Lectura del valor mínimo de la banda 2
    arrayMin2 = src.read(2)
    banda2Min = arrayMin2.min()

    # Lectura del valor máximo de la banda 2
    arrayMax2 = src.read(2)
    banda2Max = arrayMax2.max()

    # Lectura del valor mínimo de la banda 3
    arrayMin3 = src.read(3)
    banda3Min = arrayMin3.min()

    # Lectura del valor máximo de la banda 3
    arrayMax3 = src.read(3)
    banda3Max = arrayMax3.max()

    block_width = width // 4  # Dividir en 4 columnas
    block_height = height // 4  # Dividir en 4 filas

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for row in range(4):
        for col in range(4):
            x_offset = col * block_width
            y_offset = row * block_height
            output_name = f"block_{row}_{col}.tif"
            output_path = os.path.join(output_folder, output_name)

            # Construye el comando gdal_translate
            cmd = f"gdal_translate -srcwin {x_offset} {y_offset} {block_width} {block_height} -of GTiff -co PHOTOMETRIC=RGB -ot Float32 -scale_1 {banda1Min} {banda1Max} -scale_2 {banda2Min} {banda2Max} -scale_3 {banda3Min} {banda3Max} {image_path} {output_path} "
            print(cmd)

            # Ejecuta el comando en la línea de comandos
            os.system(cmd)

# Ejemplo de uso
input_image_path = "../imagen/rgbImagen32.tif"
output_folder = "../output_blocksv3"
cut_image_into_blocks(input_image_path, output_folder)



gdal_translate -srcwin 0 0 1976 2281 -of GTiff -co PHOTOMETRIC=SRGB -ot Float32 -scale_1 0.0 3383.0 -scale_2 0.0 3139.0 -scale_3 0.0 3193.0 ../imagen/rgbImagen32.tif ../output_blocksv3/block_0_0.tif 
Input file size is 7905, 9125
0...10...20...

Warning 6: 'SRGB' is an unexpected value for PHOTOMETRIC creation option of type string-select.
Warning 5: ../output_blocksv3/block_0_0.tif: PHOTOMETRIC=SRGB value not recognised, ignoring.  Set the Photometric Interpretation as MINISBLACK.


30...40...50...60...70...80...90...100 - done.
gdal_translate -srcwin 1976 0 1976 2281 -of GTiff -co PHOTOMETRIC=SRGB -ot Float32 -scale_1 0.0 3383.0 -scale_2 0.0 3139.0 -scale_3 0.0 3193.0 ../imagen/rgbImagen32.tif ../output_blocksv3/block_0_1.tif 
Input file size is 7905, 9125
0...10...20...30...

Warning 6: 'SRGB' is an unexpected value for PHOTOMETRIC creation option of type string-select.
Warning 5: ../output_blocksv3/block_0_1.tif: PHOTOMETRIC=SRGB value not recognised, ignoring.  Set the Photometric Interpretation as MINISBLACK.


40...50...60...70...80...90...100 - done.
gdal_translate -srcwin 3952 0 1976 2281 -of GTiff -co PHOTOMETRIC=SRGB -ot Float32 -scale_1 0.0 3383.0 -scale_2 0.0 3139.0 -scale_3 0.0 3193.0 ../imagen/rgbImagen32.tif ../output_blocksv3/block_0_2.tif 


...

Input file size is 7905, 9125
0...10...20...30...40...50...60...70...80...90...100 - done.
gdal_translate -srcwin 3952 6843 1976 2281 -of GTiff -co PHOTOMETRIC=SRGB -ot Float32 -scale_1 0.0 3383.0 -scale_2 0.0 3139.0 -scale_3 0.0 3193.0 ../imagen/rgbImagen32.tif ../output_blocksv3/block_3_2.tif 
Input file size is 7905, 9125
0...10...20...30...40.

Warning 6: 'SRGB' is an unexpected value for PHOTOMETRIC creation option of type string-select.
Warning 5: ../output_blocksv3/block_3_2.tif: PHOTOMETRIC=SRGB value not recognised, ignoring.  Set the Photometric Interpretation as MINISBLACK.


..50...60...70...80...90...100 - done.
gdal_translate -srcwin 5928 6843 1976 2281 -of GTiff -co PHOTOMETRIC=SRGB -ot Float32 -scale_1 0.0 3383.0 -scale_2 0.0 3139.0 -scale_3 0.0 3193.0 ../imagen/rgbImagen32.tif ../output_blocksv3/block_3_3.tif 
Input file size is 7905, 9125
0...10...20...30..

Warning 6: 'SRGB' is an unexpected value for PHOTOMETRIC creation option of type string-select.
Warning 5: ../output_blocksv3/block_3_3.tif: PHOTOMETRIC=SRGB value not recognised, ignoring.  Set the Photometric Interpretation as MINISBLACK.


.40...50...60...70...80...90...100 - done.


In [10]:
import os
import errno
import rasterio
from pathlib import Path

# Lista los archivos con os.listdir
# La ruta son los archivos tif
folder_path_tif = '/home/farix/python/imagen/tiff/'

# Cambia la ruta para guardar las imágenes png
# Cambia tiff por png, para tener una carpeta con mismo nombres de las imágenes tif
folder_path_png = folder_path_tif.replace('tiff', 'png')

# Imprime la ruta del folder png
print(folder_path_png)

# Crea folder para guardar las imágenes png
try:
    os.mkdir(folder_path_png)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

# Recorre cada archivo en la carpeta de imágenes TIF
for entry in os.listdir(folder_path_tif):
    if os.path.isfile(os.path.join(folder_path_tif, entry)):
        print(entry)
        # Para GDAL, toma los valores máximos y mínimos de cada banda en la imagen TIF
        # Con rasterio
        RGB = os.path.join(folder_path_tif, entry)
        imagen_tif = rasterio.open(RGB)

        # Lectura del valor mínimo de la banda 1
        arrayMin1 = imagen_tif.read(1)
        banda1Min = arrayMin1.min()
        
        # Lectura del valor máximo de la banda 1
        arrayMax1 = imagen_tif.read(1)
        banda1Max = arrayMax1.max()

        # Lectura del valor mínimo de la banda 2
        arrayMin2 = imagen_tif.read(2)
        banda2Min = arrayMin2.min()

        # Lectura del valor máximo de la banda 2
        arrayMax2 = imagen_tif.read(2)
        banda2Max = arrayMax2.max()

        # Lectura del valor mínimo de la banda 3
        arrayMin3 = imagen_tif.read(3)
        banda3Min = arrayMin3.min()

        # Lectura del valor máximo de la banda 3
        arrayMax3 = imagen_tif.read(3)
        banda3Max = arrayMax3.max()

        # Conversión a PNG

        # La carpeta de salida de los archivos png
        # Archivo de salida, el nombre es el mismo que el archivo de entrada con extensión PNG
        
        without_ext = os.path.splitext(entry)[0]
        file_PNG = folder_path_png + without_ext + '.png'
        print(file_PNG)

        to_png = ('gdal_translate -ot Byte -of PNG -scale_1 ' + str(banda1Min) + ' ' + str(
            banda1Max) + ' -scale_2 ' + str(banda2Min) + ' ' + str(banda2Max) + ' -scale_3 ' + str(
            banda3Min) + ' ' + str(banda3Max) + ' -a_nodata none ' + RGB + ' ' + file_PNG)
        
        print(to_png)
        
        # Ejecuta en un bash de linux para la conversión
        # os.system(to_png)

# Elimina los archivos xml generados durante la conversión
delete_xml = "rm " + folder_path_png + "*.aux.xml"
print(delete_xml)
os.system(delete_xml)


/home/farix/python/imagen/png/
rgbImagen.tif
/home/farix/python/imagen/png/rgbImagen.png
gdal_translate -ot Byte -of PNG -scale_1 0 3383 -scale_2 0 3139 -scale_3 0 3193 -a_nodata none /home/farix/python/imagen/tiff/rgbImagen.tif /home/farix/python/imagen/png/rgbImagen.png
block_1_1.tif
/home/farix/python/imagen/png/block_1_1.png
gdal_translate -ot Byte -of PNG -scale_1 310 1724 -scale_2 365 1451 -scale_3 484 1600 -a_nodata none /home/farix/python/imagen/tiff/block_1_1.tif /home/farix/python/imagen/png/block_1_1.png
block_0_1.tif
/home/farix/python/imagen/png/block_0_1.png
gdal_translate -ot Byte -of PNG -scale_1 0 1194 -scale_2 0 950 -scale_3 0 1167 -a_nodata none /home/farix/python/imagen/tiff/block_0_1.tif /home/farix/python/imagen/png/block_0_1.png
block_2_1.tif
/home/farix/python/imagen/png/block_2_1.png
gdal_translate -ot Byte -of PNG -scale_1 346 1854 -scale_2 354 1544 -scale_3 528 1547 -a_nodata none /home/farix/python/imagen/tiff/block_2_1.tif /home/farix/python/imagen/png/bloc

rm: cannot remove '/home/farix/python/imagen/png/*.aux.xml': No such file or directory


256